In [49]:
from sklearn import svm

In [52]:
import pandas as pd
import requests
import io

url = 'http://www.amlbook.com/data/zip/features.train'
s = requests.get(url).content.decode('utf-8')
df_train = pd.read_csv(io.StringIO(s), delim_whitespace=True, header=None, names=['digit', 'intensity', 'symmetry'])

url = 'http://www.amlbook.com/data/zip/features.test'
s = requests.get(url).content.decode('utf-8')
df_test = pd.read_csv(io.StringIO(s), delim_whitespace=True, header=None, names=['digit', 'intensity', 'symmetry'])

In [53]:
df_train.head()

,digit,intensity,symmetry
0,6.0,0.341092,-4.528937
1,5.0,0.444131,-5.496812
2,4.0,0.231002,-2.886750
3,7.0,0.200275,-3.534375
4,3.0,0.291936,-4.352062


In [54]:
df_test.head()

,digit,intensity,symmetry
0,9.0,0.272178,-4.847937
1,6.0,0.265133,-5.102000
2,3.0,0.335926,-2.921562
3,6.0,0.264850,-4.156625
4,6.0,0.345338,-6.718438


In [ ]:
X = df_train[['intensity', 'symmetry']]
Xt = df_test[['intensity', 'symmetry']]

In [79]:
Q = 2
C = 0.01

# 0, 2, 4, 6, 8 vs all

In [97]:
for digit in range(0,10,2):
    y = df_train.digit.apply(lambda x: 1 if x==digit else -1)
    yt = df_test.digit.apply(lambda x: 1 if x==digit else -1)
    
    poly_svc = svm.SVC(kernel='poly', degree=Q, C=C).fit(X, y)
    ein = 1-(poly_svc.predict(X) == y).mean()
    eout = 1-(poly_svc.predict(Xt) == yt).mean()
    sv = poly_svc.support_vectors_.shape[0]
    print("%s vs all - ein %.4f - eout %.4f - sv %d" %(digit, ein, eout, sv))

0 vs all - ein 0.1638 - eout 0.1769 - sv 2390
2 vs all - ein 0.1003 - eout 0.0987 - sv 1484
4 vs all - ein 0.0894 - eout 0.0997 - sv 1319
6 vs all - ein 0.0911 - eout 0.0847 - sv 1331
8 vs all - ein 0.0743 - eout 0.0827 - sv 1119


# 1, 3, 5, 7 vs all

In [98]:
for digit in range(1,10,2):
    y = df_train.digit.apply(lambda x: 1 if x==digit else -1)
    yt = df_test.digit.apply(lambda x: 1 if x==digit else -1)
    
    poly_svc = svm.SVC(kernel='poly', degree=Q, C=C).fit(X, y)
    ein = 1-(poly_svc.predict(X) == y).mean()
    eout = 1-(poly_svc.predict(Xt) == yt).mean()
    sv = poly_svc.support_vectors_.shape[0]
    print("%s vs all - ein %.4f - eout %.4f - sv %d" %(digit, ein, eout, sv))

1 vs all - ein 0.0158 - eout 0.0234 - sv 536
3 vs all - ein 0.0902 - eout 0.0827 - sv 1354
5 vs all - ein 0.0763 - eout 0.0797 - sv 1132
7 vs all - ein 0.0885 - eout 0.0732 - sv 1300
9 vs all - ein 0.0883 - eout 0.0882 - sv 1307


# 1 vs 5

In [153]:
digit=1
digit2=5
idx = df_train.digit.isin([digit, digit2])
X = df_train.ix[idx, ['intensity', 'symmetry']]
y = df_train.ix[idx].digit.apply(lambda x: 1 if x==digit else -1)

idx = df_test.digit.isin([digit, digit2])
Xt = df_test.ix[idx, ['intensity', 'symmetry']]
yt = df_test.ix[idx].digit.apply(lambda x: 1 if x==digit else -1)

In [154]:
idx = df_test.digit.isin([digit, digit2])
Xt = df_test.ix[idx, ['intensity', 'symmetry']]
yt = df_test.ix[idx].digit.apply(lambda x: 1 if x==digit else -1)

In [151]:
Q=2
for C in (0.0001, 0.001, 0.01, 0.1, 1):
    poly_svc = svm.SVC(kernel='poly', degree=Q, C=C).fit(X, y)
    ein = 1-(poly_svc.predict(X) == y).mean()
    eout = 1-(poly_svc.predict(Xt) == yt).mean()
    sv = poly_svc.support_vectors_.shape[0]
    print("C %.4f - ein %.5f - eout %.5f - sv %d" %(C, ein, eout, sv))

C 0.0001 - ein 0.02242 - eout 0.03066 - sv 510
C 0.0010 - ein 0.00705 - eout 0.01887 - sv 152
C 0.0100 - ein 0.00448 - eout 0.01887 - sv 54
C 0.1000 - ein 0.00448 - eout 0.01887 - sv 28
C 1.0000 - ein 0.00448 - eout 0.01887 - sv 25


In [152]:
Q=5
for C in (0.0001, 0.001, 0.01, 0.1, 1):
    poly_svc = svm.SVC(kernel='poly', degree=Q, C=C).fit(X, y)
    ein = 1-(poly_svc.predict(X) == y).mean()
    eout = 1-(poly_svc.predict(Xt) == yt).mean()
    sv = poly_svc.support_vectors_.shape[0]
    print("C %.4f - ein %.5f - eout %.5f - sv %d" %(C, ein, eout, sv))

C 0.0001 - ein 0.00641 - eout 0.01887 - sv 42
C 0.0010 - ein 0.00512 - eout 0.01651 - sv 28
C 0.0100 - ein 0.00448 - eout 0.01651 - sv 26
C 0.1000 - ein 0.00448 - eout 0.01651 - sv 26
C 1.0000 - ein 0.00448 - eout 0.01651 - sv 26


# Cross validation

In [ ]:
Q=2
for C in (0.0001, 0.001, 0.01, 0.1, 1):
    poly_svc = svm.SVC(kernel='poly', degree=Q, C=C).fit(X, y)
    ein = 1-(poly_svc.predict(X) == y).mean()
    eout = 1-(poly_svc.predict(Xt) == yt).mean()
    sv = poly_svc.support_vectors_.shape[0]
    print("C %.4f - ein %.5f - eout %.5f - sv %d" %(C, ein, eout, sv))

# RBF

In [223]:
digit=1
digit2=5

idx = df_train.digit.isin([digit, digit2])
X = df_train.ix[idx, ['intensity', 'symmetry']]
y = df_train.ix[idx].digit.apply(lambda x: 1 if x==digit else -1)

In [231]:
import numpy as np

RUNS = 100
C_min_list = np.zeros([RUNS, ])
e_val_min_list = np.zeros([RUNS, ])
Q=2

for i in range(RUNS):
    e_val_min = 1

    msk = np.random.rand(len(X)) < 0.9
    X_train = X[msk]
    y_train = y[msk]
    X_val = X[~msk]
    y_val = y[~msk]

    for C in (0.0001, 0.001, 0.01, 0.1, 1):
        poly_svc = svm.SVC(kernel='poly', degree=Q, C=C).fit(X_train, y_train)
        ein = 1-(poly_svc.predict(X_train) == y_train).mean()
        eout = 1-(poly_svc.predict(X_val) == y_val).mean()
        sv = poly_svc.support_vectors_.shape[0]
        if e_val < e_val_min:
            e_val_min = e_val
            C_min = C
    e_val_min_list[i] = e_val_min
    C_min_list[i] = C_min
#     print("C %.4f - ein %.5f - eval %.5f - sv %d" %(C, ein, e_val, sv))

In [225]:
for u in np.unique(C_min_list):
    print("%f %d" % (u, (C_min_list == u).sum()))

0.000100 100
